<a href="https://colab.research.google.com/github/eyaler/avatars4all/blob/master/facevidblur.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#FaceVidBlur

### Notebook by [Eyal Gruss](https://eyalgruss.com), [@eyaler](twitter.com/eyaler)

Blur (some) faces in video + optionally draw facial landmarks

Notes:
1. You should always manually inspect the output video to make sure that the faces you want to blur do not show in some frames due to miss detections.
2. For the use case of bluring *only some* of the faces, the current solution depends on having a consistent spatial ordering of faces. This may break as people move too much, get in/out of the frame, get their face occluded or turn around, as well as other face detection misses and false positives. For some of these cases the default safe_mode will try to blur extra faces to be on the safe side. I am currently working on a more robust solution.

More notebooks: [github.com/eyaler/avatars4all](https://github.com/eyaler/avatars4all)

Shortcut here: [tfi.la/blur](https://tfi.la/blur)

Something not working? Open an [issue](https://github.com/eyaler/avatars4all/issues)

If you find my work useful please consider supporting me via [GitHub Sponsors](https://github.com/sponsors/eyaler) or [PayPal](https://www.paypal.com/donate/?hosted_button_id=LNJ6F3FR79ARE)

In [ ]:
#@title Setup

import locale
locale.getpreferredencoding = lambda: 'UTF-8'

!pip install git+https://github.com/ytdl-org/youtube-dl
!pip install git+https://github.com/1adrianb/face-alignment

In [ ]:
#@title Optionally mount Google Drive (MARK CHECKBOX) { run: "auto" }
mount_google_drive = False #@param {type:"boolean"}
if mount_google_drive:
  from google.colab import drive
  drive.mount('/content/drive')
  print('path is /content/drive/MyDrive')

In [ ]:
#@title Blur faces in video

video_url = 'https://www.youtube.com/watch?v=SoAKSHcrDGg' #@param {type: 'string'}
#@markdown (leave empty to upload file - A BUTTON WILL APPEAR BELOW, or link to youtube / vimeo / video url / path to video on mounted drive [drive/MyDrive/...] / path to video on colab)
i_just_uploaded_a_file_and_i_want_to_reuse_that_instead_of_uploading_a_new_one = False #@param {type: 'boolean'}
face_order = 'left-to-right then top-to-bottom' #@param ['left-to-right then top-to-bottom', 'right-to-left then top-to-bottom', 'top-to-bottom then left-to-right', 'bottom-to-top then left-to-right']
#@markdown (one of these orderings may be more stable for your video, as order is recalculated per frame)
face_num = '' #@param {type: 'string'}
#@markdown (leave empty to blur all faces, or use face_num = 1,2,... to blur one or more faces from all the faces ordered as above, use face_num = -1,-2,... to blur all faces except these)
safe_mode = True #@param {type: 'boolean'}
#@markdown (hide neighbour faces when the number of faces changes)
discard_faces_with_longest_dimension_pixels_below = 0 #@param {type: 'integer'}
min_conf = 0.9 #@param {type: 'number'}
#@markdown (lower min_conf if faces are not detected, raise min_conf if there are false detections that you cannot get rid of by using discard_faces_with_longest_dimension_pixels_below)
radius_factor = 1 #@param {type: 'number'}
pixelization_areas_per_dimension = 1 #@param {type: 'integer'}
pixelization_colors_per_area = '1' #@param ['1', '2']
expose_eyes_mouth = False #@param {type: 'boolean'}
draw_landmarks = True #@param {type: 'boolean'}
#@markdown (uncheck this if you don't want the stupid lines over the faces)
landmarks_hex = '0000FF' #@param {type: 'string'}
start_seconds = 0 #@param {type: 'number'}
duration_seconds = 15 #@param {type: 'number'}
#@markdown (use duration_seconds = 0 for unrestricted duration)
max_width = 0 #@param {type: 'integer'}
#@markdown (use max_width = 0 for unrestricted width)
max_height = 0 #@param {type: 'integer'}
#@markdown (use max_height = 0 for unrestricted height)
copy_audio = True  #@param {type: 'boolean'}
output_filepath = 'output.mp4' #@param {type: 'string'}
#@markdown (you can specify a path to save on mounted drive [drive/MyDrive/...])

from time import time
start_time = time()

%cd /content

import os
from google.colab import files
output_filepath = os.path.normpath(output_filepath.replace('\\', '/'))
output_path, output_filename = os.path.split(output_filepath)

need_dl = True
try:
  if orig_video and video_url == save_url and (video_url or i_just_uploaded_a_file_and_i_want_to_reuse_that_instead_of_uploading_a_new_one):
    need_dl = False
except:
  pass
if need_dl:
  if not video_url:
    %cd /content/sample_data
    try:
      uploaded = files.upload()
    except Exception:
      %cd /content
      raise
    for fn in uploaded:
      orig_video = os.path.abspath(fn)
      break
    else:
      need_dl = False
    %cd /content
  elif os.path.isfile(os.path.normpath(video_url.replace('\\', '/'))):
    orig_video = os.path.abspath(os.path.normpath(video_url.replace('\\', '/')))
  elif os.path.isfile(os.path.normpath('/content/drive/MyDrive/' + video_url.replace('\\', '/'))):
    orig_video = os.path.normpath('/content/drive/MyDrive/' + video_url.replace('\\', '/'))
  else:
    orig_video = '/content/orig_video.mp4'
    !rm -f $orig_video
    !youtube-dl --no-playlist -f "bestvideo[ext=mp4][vcodec!*=av01]+bestaudio[ext=m4a]/mp4[vcodec!*=av01]/bestvideo[ext=mp4]+bestaudio[ext=m4a]/bestvideo[ext=mp4]+bestaudio/mp4" "$video_url" --merge-output-format mp4 -o $orig_video
    if not os.path.exists(orig_video):
      orig_video = '/content/orig_video.' + video_url.rsplit('.', 1)[1]
      !wget "$video_url" -O $orig_video
      if not os.path.getsize(orig_video):
        !rm -f $orig_video
    if not os.path.exists(orig_video):
      orig_video = None
      raise FileNotFoundError
  input_video = '/content/input_video' + os.path.splitext(orig_video)[-1]

need_fix_duration = True
try:
  if not need_dl and start_seconds == save_start_seconds and duration_seconds == save_duration_seconds:
    need_fix_duration = False
except:
  pass
if need_fix_duration:
  if start_seconds or duration_seconds:
    !ffmpeg -y -ss $start_seconds -t $duration_seconds -i "$orig_video" -f mp4 $input_video
  else:
    !cp "$orig_video" $input_video

save_url = video_url
save_start_seconds = start_seconds
save_duration_seconds = duration_seconds

import cv2
import face_alignment
face_detector_kwargs = {}
if min_conf < 0.5:
  face_detector_kwargs = {'filter_threshold': min_conf}
fa = face_alignment.FaceAlignment(landmarks_type=1, face_detector_kwargs=face_detector_kwargs)
import imageio.v3 as iio
from IPython.display import display, Image, Video, clear_output
import numpy as np
from skimage.draw import ellipse, polygon, polygon_perimeter

!rm -f /content/numbers.png
!rm -rf /content/in_frames
!mkdir -p /content/in_frames

face_parts = {'eyebrow1': slice(17, 22),
              'eyebrow2': slice(22, 27),
              'nose': slice(27, 31),
              'nostril': slice(31, 36),
              'eye1': slice(36, 42),
              'eye2': slice(42, 48),
              'lips': slice(48, 60),
              'teeth': slice(60, 68)
              }
step = len(landmarks_hex) // 3
landmarks_rgb = [int(landmarks_hex[i: i + step], 16) * 17**(step == 1) for i in range(0, len(landmarks_hex), step)]

face_num = face_num or None
if face_num is not None:
  face_num = [int(i) for i in face_num.split(',')]
  assert all(i > 0 for i in face_num) or all(i < 0 for i in face_num), 'face_num should be either empty or all positives or all negatives'
  neg_mode = face_num[0] < 0
  face_num = [abs(i) - 1 for i in face_num]

first_faces = ''
max_face_len_unfiltered = 0
max_face_len_below = 0
min_face_len_above = None
max_face_conf_unfiltered = 0
max_face_conf_below = 0
min_face_conf_above = 1
max_filtered_faces_in_frame = 0
frame_of_max_filtered_faces_in_frame = 0
for frame, im in enumerate(iio.imiter(input_video)):
  if min_face_len_above is None:
    min_face_len_above = max(im.shape[:2])
  landmarks, _, bboxes = fa.get_landmarks_from_image(im, return_bboxes=True)
  if bboxes is not None:
    max_face_len_unfiltered = max(max_face_len_unfiltered, *[max(x1 - x0, y1 - y0) + 1 for x0, y0, x1, y1, _ in bboxes])
    max_face_conf_unfiltered = max(max_face_conf_unfiltered, *[b[-1] for b in bboxes])
    lb = list(zip(landmarks, bboxes))

    if discard_faces_with_longest_dimension_pixels_below:
      for x0, y0, x1, y1, _ in bboxes:
        m = max(x1 - x0, y1 - y0) + 1
        if m < discard_faces_with_longest_dimension_pixels_below:
          max_face_len_below = max(max_face_len_below, m)
      lb = [(l, b) for l, b in lb if max(b[2] - b[0], b[3] - b[1]) + 1 >= discard_faces_with_longest_dimension_pixels_below]
    if lb:
      min_face_len_above = min(min_face_len_above, *[max(x1 - x0, y1 - y0) + 1 for _, (x0, y0, x1, y1, _) in lb])

    if min_conf:
      for l, b in lb:
        if b[-1] < min_conf:
          max_face_conf_below = max(max_face_conf_below, b[-1])
      lb = [(l, b) for l, b in lb if b[-1] >= min_conf]
    if lb:
      min_face_conf_above = min(min_face_conf_above, *[b[-1] for l, b in lb])

    lb = sorted(lb, key=lambda b: (b[1][0], b[1][2], b[1][1], b[1][3]) if face_order == 'left-to-right then top-to-bottom' else (-b[1][2], -b[1][0], b[1][1], b[1][3]) if face_order == 'right-to-left then top-to-bottom' else (b[1][1], b[1][3], b[1][0], b[1][2]) if face_order == 'top-to-bottom then left-to-right' else (-b[1][3], -b[1][1], b[1][0], b[1][2]))
    if len(lb) > max_filtered_faces_in_frame:
      max_filtered_faces_in_frame = len(lb)
      frame_of_max_filtered_faces_in_frame = frame
    if len(lb) > 1 and not first_faces:
      first_faces = f'{frame=}'
      face_count = len(lb)
      numbers = im.copy()
      for i, (_, (x0, y0, x1, y1, conf)) in enumerate(lb):
        cv2.putText(numbers, str(i + 1), (max(min(int(x1), im.shape[1] - 1) - 20, 0), max(min(int(y1), im.shape[0] - 1), 20)), 0, .7, (0, 255, 0), 2)
        first_faces += f'; {i + 1}: w={x1 - x0 + 1 :.1f} h={y1 - y0 + 1 :.1f} {conf=:.2f}'
      iio.imwrite('/content/numbers.png', numbers, compress_level=1)
      clear_output()
      display(Image('/content/numbers.png'))
      print(first_faces)
    if face_num is not None:
      diff = len(lb) - face_count
      if neg_mode:
        faces_to_hide = [i for i in range(len(lb)) if i not in face_num]
      else:
        faces_to_hide = face_num
      if safe_mode and diff:
        safe_faces_to_hide = []
        for n in faces_to_hide:
          start = max(min(n, n + diff), max(safe_faces_to_hide, default=-1) + 1)
          end = max(max(n, n + diff), start) + 1
          safe_faces_to_hide += list(range(start, end))
        faces_to_hide = safe_faces_to_hide
      lb = [lb[n] for n in faces_to_hide if len(lb) > n]
    orig = im.copy()
    for landmarks, (x0, y0, x1, y1, conf) in lb:
      x = (x0+x1) / 2
      y = (y0+y1) / 2
      wr = (x1-x0+1) / 2 * radius_factor
      hr = (y1-y0+1) / 2 * radius_factor
      yy, xx = ellipse(y, x, hr, wr, im.shape)
      ycrcb = cv2.cvtColor(orig[None, yy, xx], cv2.COLOR_RGB2YCrCb)[0]
      min_x = min(xx)
      spread_x = max(xx) - min_x + 1
      min_y = min(yy)
      spread_y = max(yy) - min_y + 1

      def step(z, minimum, spread):
        return (z-minimum) * pixelization_areas_per_dimension // spread % pixelization_areas_per_dimension

      areas = [[k for k in range(len(yy)) if step(yy[k], min_y, spread_y) == j and step(xx[k], min_x, spread_x) == i] for j in range(pixelization_areas_per_dimension) for i in range(pixelization_areas_per_dimension)]
      for area in areas:
        if area:
          area = np.array(area)
          if pixelization_colors_per_area == '1':
            ycrcb[area] = np.median(ycrcb[area], axis=0)
          else:
            luma = np.argsort(ycrcb[area, 0])
            dark = area[luma[: len(luma) // 2]]
            bright = area[luma[len(luma) // 2 :]]
            ycrcb[dark] = np.median(ycrcb[dark], axis=0)
            ycrcb[bright] = np.median(ycrcb[bright], axis=0)
      im[yy, xx] = cv2.cvtColor(ycrcb[None, :], cv2.COLOR_YCrCb2RGB)[0]
      if expose_eyes_mouth:
        for name, part in face_parts.items():
          if name in ('eye1', 'eye2', 'lips'):
            yy, xx = polygon(*list(zip(*landmarks[part]))[::-1], im.shape)
            im[yy, xx] = orig[yy, xx]
    if draw_landmarks:
      for landmarks, _ in lb:
        for part in face_parts.values():
          yy, xx = polygon_perimeter(*list(zip(*landmarks[part]))[::-1], im.shape)
          im[yy, xx] = landmarks_rgb
  iio.imwrite(f'/content/in_frames/frame_{frame:06d}.png', im, compress_level=1)

fps = iio.immeta(input_video)['fps']
if copy_audio:
  !ffmpeg -y -framerate $fps -thread_queue_size 0 -i /content/in_frames/frame_%06d.png -i $input_video -c:v libx264 -c:a aac -map 0:v -map 1:a? -vf "scale=min(iw\,$max_width):min(ih\,$max_height):force_original_aspect_ratio=decrease:force_divisible_by=2" -pix_fmt yuv420p -profile:v baseline -movflags +faststart "/content/$output_filename"
else:
  !ffmpeg -y -framerate $fps -thread_queue_size 0 -i /content/in_frames/frame_%06d.png -c:v libx264 -vf "scale=min(iw\,$max_width):min(ih\,$max_height):force_original_aspect_ratio=decrease:force_divisible_by=2" -pix_fmt yuv420p -profile:v baseline -movflags +faststart "/content/$output_filename"

clear_output()
if first_faces:
  display(Image('/content/numbers.png'))
  print(first_faces)
if output_path:
  !mkdir -p "$output_path"
  !cp "/content/$output_filename" "$output_filepath"
meta = iio.immeta('/content/' + output_filename)
print(f'took {(time()-start_time) / 60 :.1f} min. orig_video={orig_video}')
print(f'output_video=/content/{output_filename} w={meta["size"][0]} h={meta["size"][1]}  aspect_ratio={meta["size"][0] / meta["size"][1] :.3f} t={meta["duration"]} fps={meta["fps"]}')
print(f'max_face_len_below_{discard_faces_with_longest_dimension_pixels_below}={max_face_len_below:.1f} min_face_len_above_{discard_faces_with_longest_dimension_pixels_below}={min_face_len_above:.1f} {max_face_len_unfiltered=:.1f}')
print(f'max_face_conf_below_{min_conf}={max_face_conf_below:.2f} min_face_conf_above_{min_conf}={min_face_conf_above:.2f} {max_face_conf_unfiltered=:.2f}')
print(f'{max_filtered_faces_in_frame=} {frame_of_max_filtered_faces_in_frame=}')
print('if video does not show below, you can still download it!')
display(Video('/content/' + output_filename, embed=True, html_attributes="autoplay controls loop"))

In [ ]:
#@title Download

from google.colab import files
files.download('/content/' + output_filename)